# Práctica Guiada de Laboratorio 2 - IA Grupo 1 EPISW

### Prof. Rolando A. Maguiña Pérez

## Introducción
La presente Práctica, a realizarse el jueves 03 de Abril del 2025, trata de los denominados métodos ciegos  de búsqueda (en amplitud, en profundidad y búsqueda no determinista) y de los métodos informados (búsqueda avara, costo uniforme y A-estrella). Se pretende construir los programas que implementan los respectivos algoritmos mediante el lenguaje Python, y aplicarlos a problemas sobre grafos.

Con esa finalidad se usará parcialmente la distribución de Python llamada `AIPython` que acompaña al libro "Artificial Intelligence: foundations of computational agents" de David Poole y Alan Mackworth. Esta distribución, como los mismos autores señalan, aun no está pulida (podrían, ocasionalmente, presentarse algunos bugs) y usa principalmente el core de Python; la biblioteca que usa (casi que exclusivamente) es Matplotlib. A pesar de lo dicho, será de gran ayuda para los fines que se persiguen en el curso para el tema de búsqueda en un espacio de estados.

Al respecto, se plantearán varias preguntas las cuales se deben resolver mediante el mencionado lenguaje en un cuaderno computacional interactivo. La tarea se resolverá de forma grupal en grupos de 4 alumnos. Antes de enviarlas, los alumnos deberán verificar la corrección de sus soluciones con esta misma herramienta.

## Representación de problemas de búsqueda
Un problema de búsqueda consta de:
- un nodo de inicio
- una función *neighbors* (vecinos) que, dado un nodo, devuelve una enumeración de los arcos del nodo.
- una especificación de una meta en términos de una función booleana que toma un nodo y devuelve verdadero si el nodo es una meta
- una función heurística (opcional) que, dado un nodo, devuelve un número real no negativo. La función heurística  por defecto tiene un valor de cero.

En lo que respecta al agente de búsqueda, un nodo puede ser cualquier cosa. En los ejemplos simples, es una cadena, pero en ejemplos más complicados puede ser una tupla, un `frozenset` (un tipo muy parecido a los sets excepto que son inmutables, es decir, están congelados y no pueden ser modificados una vez inicializados) o un objeto Python.

A continuación se muestra el código para esta representación de problemas de búsqueda cuando se utiliza la distribución `AIPython`. Notar que en dicho código, `raise NotImplementedError()` es una forma de especificar que se trata de un método abstracto que debe anularse para definir un problema de búsqueda real.

In [1]:
# searchProblem.py - representations of search problems
# AIFCA Python code Version 0.9.12 Documentation at https://aipython.org
# Download the zip file and read aipython.pdf for documentation

# Artificial Intelligence: Foundations of Computational Agents https://artint.info
# Copyright 2017-2024 David L. Poole and Alan K. Mackworth
# This work is licensed under a Creative Commons
# Attribution-NonCommercial-ShareAlike 4.0 International License.
# See: https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en


import matplotlib.pyplot as plt
import random
from aipython.display import Displayable

class Search_problem(Displayable):
    """A search problem consists of:
    * a start node
    * a neighbors function that gives the neighbors of a node
    * a specification of a goal
    * a (optional) heuristic function.
    The methods must be overridden to define a search problem."""

    def start_node(self):
        """returns start node"""
        raise NotImplementedError("start_node")   # abstract method

    def is_goal(self,node):
        """is True if node is a goal"""
        raise NotImplementedError("is_goal")   # abstract method

    def neighbors(self,node):
        """returns a list (or enumeration) of the arcs for the neighbors of node"""
        raise NotImplementedError("neighbors")   # abstract method

    def heuristic(self,n):
        """Gives the heuristic value of node n.
        Returns 0 if not overridden."""
        return 0


Los `neighbors` (vecinos) son una lista de arcos. Un arco (dirigido) consta de un nodo `from_node` y un nodo `to_node`. El arco es el par `(from_node, to_node)`, pero también puede contener un valor de costo no negativo (que por defecto es 1) y puede etiquetarse con una acción.

In [3]:
class Arc(object):
    """An arc has a from_node and a to_node node and a (non-negative) cost"""
    def __init__(self, from_node, to_node, cost=1, action=None):
        self.from_node = from_node
        self.to_node = to_node
        self.action = action
        self.cost = cost
        assert cost >= 0, (f"Cost cannot be negative: {self}, cost={cost}")

    def __repr__(self):
        """string representation of an arc"""
        if self.action:
            return f"{self.from_node} --{self.action}--> {self.to_node}"
        else:
            return f"{self.from_node} --> {self.to_node}"


### Representación explícita del grafo de búsqueda
La primera representación de un problema de búsqueda es a partir de un grafo explícito (a diferencia de uno en el que se genera según se vaya necesitando).

Un **grafo explícito** consta de
- una lista o conjunto de nodos
- una lista o conjunto de arcos
- un nodo de inicio
- una lista o conjunto de nodos meta
- (opcionalmente) un diccionario que mapea un nodo a un valor heurístico para ese nodo

Como sabemos, para definir un problema de búsqueda necesitamos definir el nodo inicial, el predicado de meta, la función `neighbors` (vecinos) y la función heurística.

In [4]:
class Search_problem_from_explicit_graph(Search_problem):
    """A search problem from an explicit graph.
    """

    def __init__(self, title, nodes, arcs, start=None, goals=set(), hmap={},
                     positions=None, show_costs = True):
        """ A search problem consists of:
        * list or set of nodes
        * list or set of arcs
        * start node
        * list or set of goal nodes
        * hmap: dictionary that maps each node into its heuristic value.
        * positions: dictionary that maps each node into its (x,y) position
        * show_costs is used for show()
        """
        self.title = title
        self.neighs = {}
        self.nodes = nodes
        for node in nodes:
            self.neighs[node]=[]
        self.arcs = arcs
        for arc in arcs:
            self.neighs[arc.from_node].append(arc)
        self.start = start
        self.goals = goals
        self.hmap = hmap
        if positions is None:
            self.positions = {node:(random.random(),random.random()) for node in nodes}
        else:
            self.positions = positions
        self.show_costs = show_costs


    def start_node(self):
        """returns start node"""
        return self.start

    def is_goal(self,node):
        """is True if node is a goal"""
        return node in self.goals

    def neighbors(self,node):
        """returns the neighbors of node (a list of arcs)"""
        return self.neighs[node]

    def heuristic(self,node):
        """Gives the heuristic value of node n.
        Returns 0 if not overridden in the hmap."""
        if node in self.hmap:
            return self.hmap[node]
        else:
            return 0

    def __repr__(self):
        """returns a string representation of the search problem"""
        res=""
        for arc in self.arcs:
            res += f"{arc}.  "
        return res



### Rutas
Un agente de búsqueda devolverá una ruta desde el nodo inicial hasta un nodo meta. Una lista de Python no es una representación adecuada para una ruta, ya que muchos algoritmos de búsqueda consideran varias rutas a la vez y estas rutas deben compartir partes iniciales de la ruta. Si quisiéramos hacer esto con listas de Python, necesitaríamos seguir copiando la lista, lo que puede resultar costoso si la lista es larga. Aquí se utiliza una representación alternativa en términos de una estructura de datos recursiva que puede compartir subpartes.

Un camino es:
- un nodo (que representa un camino de longitud 0) o
- una ruta, `initial` y un arco, donde el `from_node` del arco es el nodo al final de `initial`.

Estos casos se distinguen en el siguiente código haciendo `arc=None` si la ruta tiene longitud 0, en cuyo caso `initial` es el nodo de la ruta. Tenga en cuenta que sólo utilizamos la forma más básica de Python `yield` para las enumeraciones (Sección 1.5.4 del manual).


In [5]:
class Path(object):
    """A path is either a node or a path followed by an arc"""

    def __init__(self,initial,arc=None):
        """initial is either a node (in which case arc is None) or
        a path (in which case arc is an object of type Arc)"""
        self.initial = initial
        self.arc=arc
        if arc is None:
            self.cost=0
        else:
            self.cost = initial.cost+arc.cost

    def end(self):
        """returns the node at the end of the path"""
        if self.arc is None:
            return self.initial
        else:
            return self.arc.to_node

    def nodes(self):
        """enumerates the nodes for the path.
        This enumerates the nodes in the path from the last elements backwards.
        """
        current = self
        while current.arc is not None:
            yield current.arc.to_node
            current = current.initial
        yield current.initial

    def initial_nodes(self):
        """enumerates the nodes for the path before the end node.
        This calls nodes() for the initial part of the path.
        """
        if self.arc is not None:
            yield from self.initial.nodes()

    def __repr__(self):
        """returns a string representation of a path"""
        if self.arc is None:
            return str(self.initial)
        elif self.arc.action:
            return f"{self.initial}\n   --{self.arc.action}--> {self.arc.to_node}"
        else:
            return f"{self.initial} --> {self.arc.to_node}"



### EJM. Problemas de Búsqueda

In [ ]:
from IPython.display import Image
i = Image(filename='D:\\Cursos\\IA\\Busq en Espacio Estado\\2024-1\\Fig1.jpg')
i

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

                Figura 1

El primer problema de búsqueda que analizaremos es uno con 5 nodos donde la ruta de menor costo es aquella con muchos arcos (ver Figura 1).

In [8]:
problem1 = Search_problem_from_explicit_graph('Problem 1', {'A','B','C','D','G'},
   [Arc('A','B'), Arc('A','C'), Arc('B','D'), Arc('B','G'), Arc('C','B'), Arc('C','D'), Arc('D','G')],
   start = 'A',
   goals = {'G'},
   positions={'A': (0, 1), 'B': (0.5, 0.5), 'C': (0,0.5), 'D': (0.5,0), 'G': (1,0)})

### Agente de búsqueda
A un agente de búsqueda `Searcher` para un problema se le puede preguntar repetidamente cuál es la siguiente ruta. Para resolver un problema, se puede construir un objeto `Searcher` para dicho problema y luego solicitar repetidamente la siguiente ruta mediante  `search`. Si no hay más rutas, se devuelve `None`.

In [9]:
# searchGeneric.py - Generic Searcher, including depth-first and A*
# AIFCA Python code Version 0.9.12 Documentation at https://aipython.org
# Download the zip file and read aipython.pdf for documentation

# Artificial Intelligence: Foundations of Computational Agents https://artint.info
# Copyright 2017-2024 David L. Poole and Alan K. Mackworth
# This work is licensed under a Creative Commons
# Attribution-NonCommercial-ShareAlike 4.0 International License.
# See: https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en



class Searcher(Displayable):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0
        self.add_to_frontier(Path(problem.start_node()))
        super().__init__()

    def initialize_frontier(self):
        self.frontier = []

    def empty_frontier(self):
        return self.frontier == []

    def add_to_frontier(self,path):
        self.frontier.append(path)

    def search(self):
        """returns (next) path from the problem's start node
        to a goal node.
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            self.path = self.frontier.pop()
            self.num_expanded += 1
            if self.problem.is_goal(self.path.end()):    # solution found
                self.solution = self.path   # store the solution found
                self.display(1, f"Solution: {self.path} (cost: {self.path.cost})\n",
                    self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                return self.path
            else:
                self.display(4,f"Expanding: {self.path} (cost: {self.path.cost})")
                neighs = self.problem.neighbors(self.path.end())
                self.display(2,f"Expanding: {self.path} with neighbors {neighs}")
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(self.path,arc))
                self.display(3, f"New frontier: {[p.end() for p in self.frontier]}")

        self.display(0,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")

# Depth-first search for problem1; do the following:
# searcher1 = Searcher(searchExample.problem1)
# searcher1.search()  # find first solution
# searcher1.search()  # find next solution (repeat until no solutions)
# searcher_sdg = Searcher(searchExample.simp_delivery_graph)
# searcher_sdg.search()  # find first or next solution



Notar que esto invierte los neighbors (vecinos) de modo que implementa la búsqueda en profundidad de una manera intuitiva (expandiendo primero el primer vecino). La llamada a  `list` es para el caso en el que se generan los vecinos (y aún no están en una lista). Es posible que no se requiera invertir los neighbors (vecinos) para otros métodos. Las llamadas a `reversed` y a `list` se pueden eliminar, y el algoritmo aún implementa la búsqueda en profundidad.

Para utilizar la búsqueda en profundidad para encontrar múltiples rutas para el `problem1`, ejecute el código contenido en la celda que viene a continuación; puede seguir buscando las siguientes soluciones hasta que no haya más:

In [10]:
# Depth-first search for problem1; do the following:
searcher1 = Searcher(problem1)
searcher1.search() # find first solution
# searcher1.search() # find next solution (repeat until no solutions)

Solution: A --> B --> D --> G (cost: 3)
 4 paths have been expanded and 2 paths remain in the frontier


A --> B --> D --> G

In [11]:
searcher1.search()

Solution: A --> B --> G (cost: 2)
 5 paths have been expanded and 1 paths remain in the frontier


A --> B --> G

### Frontera de caminos
Como sabemos, un algoritmo genérico para la búsqueda de una ruta solución en un grafo llama a procedimientos que pueden codificarse para implementar varias estrategias de búsqueda.

La idea intuitiva detrás del algoritmo de búsqueda genérico, dado un grafo, un nodo inicial y un predicado de meta, es explorar rutas de forma incremental desde el nodo inicial. Esto se hace manteniendo una `frontera` (o franja) de caminos desde el nodo inicial; contiene todos los caminos que podrían formar segmentos iniciales de caminos desde el nodo inicial hasta un nodo meta (ver figura en celda siguiente, donde la `frontera` es el conjunto de caminos hacia los nodos sombreados en gris). Inicialmente, la `frontera` contiene el camino trivial que contiene sólo el nodo inicial y ningún arco. A medida que avanza la búsqueda, la `frontera` se expande hacia los nodos inexplorados hasta que se encuentra un nodo meta. Mediante una adecuada implementación de la `frontera` se pueden obtener diferentes estrategias de búsqueda.

In [ ]:
from IPython.display import Image
i = Image(filename='E:\\Curso EPISW\\IA 2025-1\\BEE\\frontera.png')
i

###  Frontera como cola de prioridad
En muchos de los algoritmos de búsqueda, como A* y otros métodos de búsqueda del tipo primero el mejor, la frontera se implementa como una cola de prioridad. El código mostrado a continuación utiliza las implementaciones de cola de prioridad integradas de Python, `heapq`.

Siguiendo el ejemplo de la documentación de Python, https://docs.python.org/3/library/heapq.html - o su versión en castellano  [heapq — Algoritmo de colas montículos (heap)](https://docs.python.org/es/3.10/library/heapq.html) - una frontera es una lista de tripletas. El primer elemento de cada tripleta es el valor a minimizar. El segundo elemento es un índice único que especifica el orden en que se agregaron los elementos a la cola y el tercer elemento es la ruta que está en la cola. El uso del índice único garantiza que la implementación de la cola de prioridad no compare rutas; no está definido si una ruta es menor que otra. También nos permite controlar qué tipo de búsqueda (p.ej., primero en profundidad o primero en amplitud) se produce cuando el valor a minimizar no proporciona una siguiente ruta única.

La variable `frontier index` es el número total de elementos de la frontera que se han creado. Además de utilizarse como índice único, es útil para las estadísticas, especialmente en relación con el tamaño actual de la frontera.

In [14]:
import heapq # part of the Python standard library

class FrontierPQ(object):
    """A frontier consists of a priority queue (heap), frontierpq, of
       (value, index, path) triples, where
    * value is the value we want to minimize (e.g., path cost + h).
    * index is a unique index for each element
    * path is the path on the queue
    Note that the priority queue always returns the smallest element.
    """

    def __init__(self):
        """constructs the frontier, initially an empty priority queue
        """
        self.frontier_index = 0 # the number of items added to the frontier
        self.frontierpq = [] # the frontier priority queue

    def empty(self):
        """is True if the priority queue is empty"""
        return self.frontierpq == []

    def add(self, path, value):
        """add a path to the priority queue
        value is the value to be minimized"""
        self.frontier_index += 1 # get a new unique index
        heapq.heappush(self.frontierpq,(value, -self.frontier_index, path))

    def pop(self):
        """returns and removes the path of the frontier with minimum value.
        """
        (_,_,path) = heapq.heappop(self.frontierpq)
        return path


Los sgtes métodos se utilizan para buscar e imprimir información sobre la frontera.

In [15]:
def count(self,val):
       """returns the number of elements of the frontier with value=val"""
       return sum(1 for e in self.frontierpq if e[0]==val)

def __repr__(self):
    """string representation of the frontier"""
    return str([(n,c,str(p)) for (n,c,p) in self.frontierpq])
def __len__(self):
       """length of the frontier"""
       return len(self.frontierpq)

def __iter__(self):
       """iterate through the paths in the frontier"""
       for (_,_,path) in self.frontierpq:
           yield path

## Búsqueda A*
Para una búsqueda A*, la frontera se implementa utilizando la clase `FrontierPQ` creada en la celda anterior.

In [16]:
class AStarSearcher(Searcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """

    def __init__(self, problem):
        super().__init__(problem)

    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    def empty_frontier(self):
        return self.frontier.empty()

    def add_to_frontier(self,path):
        """add path to the frontier with the appropriate cost"""
        value = path.cost+self.problem.heuristic(path.end())
        self.frontier.add(path, value)


## Ejercicios
### Ejercicio 1
- Estudie detenidamente la versión del algoritmo de búsqueda presentada (usa búsqueda en profundidad por defecto) en este documento.
- Aplique dicho algoritmo para obtener la ruta solución para el camino s-m en el grafo mostrado en la figura A.
- Muestre la visualización gráfica del grafo de búsqueda. Use para tal efecto el código disponible en el manual de la distribución AIPython. Agregue las líneas de código que se requieran.

In [ ]:
from IPython.display import Image
i = Image(filename='C:\\Users\\rolan\\Cuads computs\\IA 2021-1\\arbol1.jpg')
i

          Figura A

### Ejercicio 2
- A partir de la versión del algoritmo de búsqueda presentada (usa búsqueda en profundidad por defecto), implemente ahora el algoritmo de búsqueda no determinista. Documente su código.
- Tomando como referencia el grafo mostrado en la figura 1, aplique dicho algoritmo. Explique razonadamente su respuesta.

### Ejercicio 3
- Tomando como referencia el problema denominado `El Viaje por Andalucía` (descrito en este mismo documento), aplique el algoritmo A* al problema de determinar una ruta Sevilla-Almería. Use la implementación mostrada en este cuaderno computacional interactivo, efectuando (cuando se requiera) las modificaciones al código requeridas para su adaptación.        (6 puntos)
- Enseguida use el código que ha desarrollado para determinar otra ruta, tal como Almería-Huelva.  (1 punto)

### Ejercicio 4
- Haga lo mismo (estudiar conceptos y el código *original* mostrados en celdas anteriores) para implementar ahora el algoritmo de búsqueda por costo uniforme para resolver el problema de determinar una ruta Sevilla-Almería. (5 puntos)
- Enseguida use el código desarrollado por usted para determinar otra ruta, tal como Almería-Huelva. (1 punto)

**Nota**: cuando se pide efectuar modificaciones en un código base quiere decir que se pueden definir funciones e insertarlas  adecuadamente en alguna parte del mismo, usando funciones ya definidas, eliminar algunas líneas de código, modificando el código innecesario, entre otras acciones.

## EL VIAJE POR LAS CIUDADES DE ANDALUCÍA
Supongamos nos encontramos en una capital de la región de España denominada Andalucía, p.e. Sevilla y deseamos ir a otra capital andaluza, p.e. AlmerÍa. Se sabe que los autobuses sólo van de cada capital a sus vecinas. Enseguida se muestra un mapa de dicha región y las coordenadas geográficas de las ciudades que pertenecen a ella:

In [ ]:
from IPython.display import Image
i = Image(filename='D:\\Cursos\\IA\\Python\\2021-1\\andalucia-distancias.png')
i

### Almacenamiento de coordenadas de ciudades
Usaremos un diccionario para almacenar la información sobre las coordenadas de c/u de las ciudades.

In [18]:
ciudades = {"almeria": [409.5,93], "granada":[309,127.5], "malaga":[232.5,75], "cadiz":[63,57], "huelva":[3,139.5], "sevilla":[90,153], "cordoba":[198,207], "jaen":[295.5,192]}
print(ciudades)

{'almeria': [409.5, 93], 'granada': [309, 127.5], 'malaga': [232.5, 75], 'cadiz': [63, 57], 'huelva': [3, 139.5], 'sevilla': [90, 153], 'cordoba': [198, 207], 'jaen': [295.5, 192]}


### Definición de función distancia
Definimos una función distancia que, dadas dos ciudades, devuelve la distancia Euclidiana entre ellas. Usa las coordenadas (abcisa,ordenada) de las ciudades en dicho cálculo; las coordenadas se obtienen aplicando el método get(keyN) al diccionario ciudades, donde N={1,2} corresponden a la ciudad 1 ó 2.

In [19]:
import math

def distancia(key1,key2):
    abcisa1,ordenada1 = ciudades.get(key1)
    abcisa2,ordenada2 = ciudades.get(key2)
    return math.sqrt((abcisa1 - abcisa2)**2 + (ordenada1 - ordenada2)**2)

## Instrucciones para el envío de la solución
La solución de la "Práctica de Laboratorio 2 - IA 2025-1 Sección 1 EPISW" podrá enviarse al correo electrónico rmaguinacursos@gmail.com, hasta las 23:59 h del Sábado 06 de Abril del 2025, en un archivo con extensión .ipynb.

El documento deberá tener las sgtes características:

- Nombre del archivo:

       solPGL2_IA_2025-1_G1_EPISW_nombre-apellidos-alumno1_..._nombre-apellidos-alumno4.ipynb

- Todas las preguntas de la Práctica deben responderse en un cuaderno computacional interactivo (**Sugerencia**: obtener una copia de este documento y desarrollar en éllas respectivas soluciones); la solución a cada pregunta debe registrarse en una celda debajo del planteamiento de la misma, mencionando explícitamente como subtítulo: "Solución del ejercicio n", donde "n" corresponde al número del ejercicio.